In [ ]:
import pandapower as pp
from pandapower import runpp
from pandapower.plotting import simple_plotly, pf_res_plotly
import pandapower.networks as networks
from citylearn import CityLearn
from gridlearn import GridLearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import random

In [ ]:
np.random.seed(0)
climate_zone = 1
data_path = Path("data/Climate_Zone_"+str(climate_zone))
building_attributes = data_path / 'building_attributes.json'
weather_file = data_path / 'weather_data.csv'
solar_profile = data_path / 'solar_generation_1kW.csv'
building_state_actions = 'buildings_state_action_space.json'
building_ids = ["Building_1","Building_2","Building_3","Building_4","Building_5","Building_6","Building_7","Building_8","Building_9","Building_10"]
# building_ids = ["Building_1","Building_4","Building_5","Building_6"] # only building types with pv
objective_function = ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption', 'system_losses']
my_grid = GridLearn(data_path, building_attributes, weather_file, solar_profile, building_ids, 6, buildings_states_actions = building_state_actions, cost_function = objective_function, verbose=1, n_buildings_per_bus=1)

In [ ]:
observations_spaces, actions_spaces = my_grid.get_state_action_spaces()

# Simulation without energy storage
my_grid.reset()
done = False
# while not done:
for ts in range(6):
    # for the do nothing action:
    _, rewards, done, _ = my_grid.step([[0 for _ in range(actions_spaces[i].shape[0])] for i in range(len(my_grid.buildings))])
    
#     # for randomized actions:
#     _, rewards, done, _ = my_grid.step([actions_spaces[i].sample() for i in range(len(my_grid.buildings))])
    

In [ ]:
pf_res_plotly(my_grid.net)
print()

In [ ]:
my_grid.plot_all()

In [ ]:
my_grid.cost()

In [1]:
# Run this again after editing submodules so Colab uses the updated versions
from citylearn import CityLearn
from gridlearn import GridLearn
import matplotlib.pyplot as plt
from pathlib import Path
from agent import RL_Agents_Coord
import numpy as np                                                                                                                                                                                      
import csv
import time
import re
import pandas as pd
import torch
from joblib import dump, load

# Load environment
climate_zone = 1
data_path = Path("data/Climate_Zone_"+str(climate_zone))
building_attributes = data_path / 'building_attributes.json'
weather_file = data_path / 'weather_data.csv'
solar_profile = data_path / 'solar_generation_1kW.csv'
building_state_actions = 'buildings_state_action_space.json'
building_id = ["Building_1","Building_2","Building_3","Building_4","Building_5","Building_6","Building_7","Building_8","Building_9"]
objective_function = ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','quadratic','system_losses']

print("Initializing the grid...")
# Contain the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
# Can be obtained using observations_spaces[i].low or .high
env = GridLearn(data_path, building_attributes, weather_file, solar_profile, building_id, 6, buildings_states_actions = building_state_actions, simulation_period = (0,4), cost_function = objective_function, verbose=1, n_buildings_per_bus=1)

# Hyperparameters
bs = 256
tau = 0.005
gamma = 0.99
lr = 0.0003
hid = [256,256]

n_episodes = 4

print("Initializing the agents...")
# Instantiating the control agent(s)
agents = RL_Agents_Coord(env, discount = gamma, batch_size = bs, replay_buffer_capacity = 1e5, regression_buffer_capacity = 12*8760, tau=tau, lr=lr, hidden_dim=hid, start_training=8760*3, exploration_period = 8760*3+1,  start_regression=8760, information_sharing = True, pca_compression = .95, action_scaling_coef=0.5, reward_scaling = 5., update_per_step = 1, iterations_as = 2)

print("Starting the experiment...")
# The number of episodes can be replaces by a stopping criterion (i.e. convergence of the average reward)
start = time.time()
for e in range(n_episodes): 
    is_evaluating = (e > 7) # Evaluate deterministic policy after 7 epochs
    rewards = []
    state = env.reset()
    done = False

    j = 0
    action, coordination_vars = agents.select_action(state, deterministic=is_evaluating)    
    while not done:
        next_state, reward, done, _ = env.step(action)
        action_next, coordination_vars_next = agents.select_action(next_state, deterministic=is_evaluating)
        agents.add_to_buffer(state, action, reward, next_state, done, coordination_vars, coordination_vars_next)

        state = next_state
        coordination_vars = coordination_vars_next
        action = action_next

    print('Loss -',env.cost(), 'Simulation time (min) -',(time.time()-start)/60.0)

Initializing the grid...
Initializing the agents...
Starting the experiment...


/Users/aislingpigott/Documents/CityLearn/citylearn.py:560: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.state.append(np.array(s))


Cumulated reward: 20.67531443364198


/Users/aislingpigott/Documents/CityLearn/citylearn.py:560: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.state.append(np.array(s))


Loss - {'ramping': 0.029705783, '1-load_factor': 0.09900749118339175, 'average_daily_peak': 0.16063409, 'peak_demand': 0.16063409, 'net_electricity_consumption': 0.3178293, 'system_losses': 0.3157936940038272, 'quadratic': 0.06935744, 'total': 0.16470884148301376} Simulation time (min) - 1.296626083056132
Cumulated reward: 20.615952785804332
Loss - {'ramping': 0.031054115, '1-load_factor': 0.13040818277411337, 'average_daily_peak': 0.16418128, 'peak_demand': 0.16418128, 'net_electricity_consumption': 0.31924835, 'system_losses': 0.3172644005620347, 'quadratic': 0.07006589, 'total': 0.17091478489687237} Simulation time (min) - 1.623163398106893
Cumulated reward: 20.261830500340555
Loss - {'ramping': 0.054566707, '1-load_factor': 0.17830275783391633, 'average_daily_peak': 0.16284174, 'peak_demand': 0.16284174, 'net_electricity_consumption': 0.30817264, 'system_losses': 0.3061806387596862, 'quadratic': 0.06545615, 'total': 0.17690891053476107} Simulation time (min) - 1.9505075136820476
Cu

In [2]:
env.system_losses

[0.785291186515646, 0.8366420820418948, 0.7307344021784503, 0.673332128442126]

In [ ]:
list(env.buildings.keys())